In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}


In [3]:
from splinter import Browser
import time
import requests
from PIL import Image
#browser = Browser('chrome', executable_path, headless=False)

In [5]:
browser.visit('http://google.com')
browser.fill('q', 'splinter - python acceptance testing for web applications')


In [6]:
browser.find_by_name('btnK').click()


In [8]:
if browser.is_text_present('splinter.readthedocs.io'):
    print("Yes, found it! :)")
else:
    print("No, didn't find it :(")

Yes, found it! :)


In [9]:
browser.quit()

In [10]:
from splinter import Browser

browser = Browser('chrome', executable_path, headless=False) # defaults to firefox
browser.visit('http://google.com')
browser.fill('q', 'Dogs')
browser.find_by_name('btnK').click()




#browser.quit()

In [4]:
def scroll_to_end(driver,sleep_interactions):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_interactions)

        #selenium.time.sleep(sleep_between_interactions)    

In [7]:
def fetch_images(path, query,s_interactions,max_links_to_fetch):

    #open chrome driver
    wd = Browser('chrome', path, headless=False) # defaults to firefox

    #create query string
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    
    #search for the query
    wd.visit(search_url.format(q=query))

    
    #parameter set up
    image_urls = set()
    image_count = 0
    results_start = 0
    print("working")

    while image_count < max_links_to_fetch:
        scroll_to_end(wd,s_interactions)

        # get all image thumbnail results
        thumbnail_results = wd.find_by_css("img.Q4LuWd")
        number_results = len(thumbnail_results)
        image_count += 1
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        print(image_count)

        for img in thumbnail_results[results_start:max_links_to_fetch]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(s_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_by_css('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image['src'] and 'http' in actual_image['src']:
                    image_urls.add(actual_image['src'])

    #         image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls
    #wd.quit()

In [8]:
fetch_images(executable_path,"Dogs",60,1)

working
Found: 100 search results. Extracting links from 0:100
1
Found: 1 image links, done!


{'https://i.ytimg.com/vi/MPV2METPeJU/maxresdefault.jpg'}

In [ ]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)